**Libraries**

In [ ]:
# Data manipulation
import numpy as np         # Fundamental package for numerical computing
import pandas as pd        # Data manipulation and analysis

# File and system operations
import os                  # Operating system operations
import pickle              # Serialization of Python objects

# Visualization
import matplotlib.pyplot as plt  # Creating visualizations
import seaborn as sns            # Statistical data visualization

# Machine learning and deep learning
import torch               # Deep learning framework
import torch.nn as nn      # Neural network modules
import torch.nn.functional as F  # Neural network functional operations
import torch.optim as optim     # Optimization algorithms
from torch.utils.data import DataLoader, SubsetRandomSampler, TensorDataset, random_split, ConcatDataset  # Data handling

# Computer vision
import torchvision         # Computer vision library
import torchvision.transforms as transforms  # Image transformations
import torchvision.datasets as datasets      # Standard datasets

# Model summary
from torchsummary import summary  # Summarizing PyTorch models

# Metrics and model selection
from sklearn.metrics import confusion_matrix  # Confusion matrix
from sklearn.model_selection import KFold      # k-fold cross-validation
from sklearn.model_selection import StratifiedKFold

**Load Dataset**

In [ ]:
#Access files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Paths to training and test folders
train_folder_path = '/content/drive/.../train'
test_folder_path = '/content/drive/M.../test'##

batch_size   = 32
num_epochs   = 15
k_folds      = 5
lr           = 1e-3
momentum     = 0.9
weight_decay = 1e-3
        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

**Data Preprocessing**

In [ ]:
# %% [4] Transforms & Datasets
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2,0.2,0.2,0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
])
valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
])

# train_set used for cross-validation; test_set held out for final evaluation
train_set = datasets.ImageFolder(train_folder_path, transform=train_transform)
test_set  = datasets.ImageFolder(test_folder_path,  transform=valid_transform)

classes = train_set.classes
print("Classes:", classes)

# Extract targets array for StratifiedKFold
targets = np.array([sample[1] for sample in train_set.samples])

skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)


In [ ]:
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct = 0.0, 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        correct      += (outputs.argmax(1) == labels).sum().item()
    avg_loss = running_loss / len(loader.sampler)
    avg_acc  = correct      / len(loader.sampler) * 100
    return avg_loss, avg_acc

def valid_epoch(model, loader, criterion):
    model.eval()
    running_loss, correct = 0.0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)
            correct      += (outputs.argmax(1) == labels).sum().item()
    avg_loss = running_loss / len(loader.sampler)
    avg_acc  = correct      / len(loader.sampler) * 100
    return avg_loss, avg_acc


**Model Training**

In [ ]:
# %% [6] Cross-Validation + Fine-Tuned AlexNet
fold_results = {}

for fold, (train_idx, val_idx) in enumerate(skf.split(np.arange(len(train_set)), targets), 1):
    print(f"\n--- Fold {fold}/{k_folds} ---")

    # Samplers & Loaders
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler   = SubsetRandomSampler(val_idx)

    train_loader = DataLoader(train_set, batch_size=batch_size, sampler=train_sampler, num_workers=2)
    val_loader   = DataLoader(train_set, batch_size=batch_size, sampler=val_sampler,   num_workers=2)

    # Load pretrained AlexNet & freeze features
    model = models.alexnet(pretrained=True)
    for param in model.features.parameters():
        param.requires_grad = False

    # Replace classifier head
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, len(classes))
    nn.init.xavier_uniform_(model.classifier[6].weight)
    model = model.to(device)

    # Show summary once (on first fold)
    if fold==1:
        summary(model, (3,224,224))



    # Loss & Optimizer (only classifier params)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                          lr=lr, momentum=momentum, weight_decay=weight_decay)

    # Train/Validate
    history = {'train_loss':[], 'train_acc':[], 'val_loss':[], 'val_acc':[]}
    for epoch in tqdm(range(1, num_epochs+1), desc=f"Fold {fold}"):
        tr_loss, tr_acc = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_acc = valid_epoch(model, val_loader, criterion)

        history['train_loss'].append(tr_loss)
        history['train_acc'].append(tr_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print(f"Epoch {epoch}/{num_epochs} — "
              f"Train: {tr_loss:.4f}, {tr_acc:5.2f}% | "
              f"Val:   {val_loss:.4f}, {val_acc:5.2f}%")

    fold_results[f"fold{fold}"] = history


In [ ]:
# Save the model to a file
torch.save(model.state_dict(), 'model.pth')

In [ ]:
# Visualize cross-validation results
import matplotlib.pyplot as plt

# Plot loss curves for each fold

plt.figure(figsize=(10,5))
for fold, history in fold_results.items():
    plt.plot(history['train_loss'], label=f"{fold} Train Loss")
    plt.plot(history['val_loss'],   label=f"{fold} Val Loss")

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs. Validation Loss per Fold")
plt.legend(fontsize='small')
plt.grid(True)
plt.show()

# Plot accuracy curves for each fold
plt.figure(figsize=(10,5))
for fold, history in fold_results.items():
    plt.plot(history['train_acc'], label=f"{fold} Train Acc")
    plt.plot(history['val_acc'],   label=f"{fold} Val Acc")

plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Training vs. Validation Accuracy per Fold")
plt.legend(fontsize='small')
plt.grid(True)
plt.show()

In [ ]:

# %% [X+2] Average Performance Across Folds
# Stack histories into arrays
train_losses = np.stack([h['train_loss'] for h in fold_results.values()], axis=0)
val_losses   = np.stack([h['val_loss']   for h in fold_results.values()], axis=0)
train_accs   = np.stack([h['train_acc'] for h in fold_results.values()], axis=0)
val_accs     = np.stack([h['val_acc']   for h in fold_results.values()], axis=0)

# Compute epoch-wise means
train_loss_avg = train_losses.mean(axis=0)
val_loss_avg   = val_losses.mean(axis=0)
train_acc_avg  = train_accs.mean(axis=0)
val_acc_avg    = val_accs.mean(axis=0)

# Plot averaged loss
plt.figure(figsize=(10,5))
plt.plot(train_loss_avg, label='Avg Train Loss')
plt.plot(val_loss_avg,   label='Avg Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Average Loss Across Folds')
plt.legend()
plt.grid(True)
plt.show()

# Plot averaged accuracy
plt.figure(figsize=(10,5))
plt.plot(train_acc_avg, label='Avg Train Acc')
plt.plot(val_acc_avg,   label='Avg Val Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Average Accuracy Across Folds')
plt.legend()
plt.grid(True)
plt.show()

# Print final-epoch averages
print(f"Final Avg Train Loss: {train_loss_avg[-1]:.4f}")
print(f"Final Avg Val   Loss: {val_loss_avg[-1]:.4f}")
print(f"Final Avg Train Acc : {train_acc_avg[-1]:.2f}%")
print(f"Final Avg Val   Acc : {val_acc_avg[-1]:.2f}%")

**MODEL EVALUATION**

In [ ]:

test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

# Build confusion matrix tensor
cm_tensor = torch.zeros(len(classes), len(classes), dtype=torch.int64)

correct, total = 0, 0
model.eval()
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        preds = model(imgs).argmax(dim=1)
        correct += (preds == labels).sum().item()
        total   += labels.size(0)
        for t, p in zip(labels.view(-1), preds.view(-1)):
            cm_tensor[t, p] += 1

test_acc = correct / total * 100
print(f"\nTest Accuracy: {test_acc:.2f}%")
print("Confusion Matrix (rows=true, cols=pred):\n", cm_tensor)


In [ ]:
plt.figure(figsize=(6, 5))
sns.heatmap(cm_tensor.cpu().numpy(), annot=True, fmt='d',
            xticklabels=classes, yticklabels=classes, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# %% [X+5] Compute Precision, Recall, F1-Score
tp = cm_tensor[1,1].item()
tn = cm_tensor[0,0].item()
fp = cm_tensor[0,1].item()
fn = cm_tensor[1,0].item()

precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
f1_score  = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

metrics_df = pd.DataFrame({
    'Accuracy': [test_acc],
    'Precision': [precision * 100],
    'Recall':    [recall    * 100],
    'F1-Score':  [f1_score  * 100]
}, index=['Overall'])

print("\nClassification Metrics:")
print(metrics_df.round(2))